**Préparation de nos données pour entrée dans le TUTO, à partir du formatage spacy fournis dans le TUTO et adapté à nos données**

In [ ]:
import json

with open('combined_data_output.json', 'r') as f:
    data = json.load(f)

print(data['examples'][0])

In [ ]:
# Adaptation des classes en fonction des annotations de ton dataset
training_data = {'classes' : ['PER', 'LOC', 'ORG', 'MISC'], 'annotations' : []}  # Modifier ces classes selon ton jeu de données

# Parcours des exemples
for example in data['examples']:
    temp_dict = {}
    temp_dict['text'] = example['content']  # Le contenu du texte
    temp_dict['entities'] = []  # Liste vide pour les entités à annoter

    # Parcours des annotations de chaque exemple
    for annotation in example['annotations']:
        start = annotation['start']  # Position de début de l'entité
        end = annotation['end']      # Position de fin de l'entité
        label = annotation['tag_name'].upper()  # Label de l'entité, convertir en majuscules si nécessaire
        temp_dict['entities'].append((start, end, label))  # Ajouter l'entité à la liste

    # Ajouter l'exemple avec ses entités dans le training_data
    training_data['annotations'].append(temp_dict)

# Afficher le premier exemple avec ses annotations
print(training_data['annotations'][0])

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("fr") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [ ]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="expand")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object